In [23]:
import pandas as pd
import numpy as np

In [24]:
df = pd.read_csv("Z:/00_ETL/Collaborative_filtering/data_4.txt", sep = "\t")
df.columns = ['UserID','MovieID','content_type','content_length_seconds_total','pause_count',
              'seconds_viewed','seconds_paused','offer_group_desc']
              

In [25]:
movies_df = pd.read_csv("../Collaborative_filtering/data/movies_df.txt", sep = '\t', header = None, encoding = 'ISO-8859-1')
movies_df.columns = ['Title','MovieID']
movies_df.head()

,Title,MovieID
0,Switch - Ein mörderischer Tausch,B00P5GGIQY
1,Varsity Blues,B00I8QYZDQ
2,Holy Flying Circus,B01B1M5HRG
3,Tuscan Wedding - Hochzeit auf Italienisch [dt....,B01E85784S
4,Just Married [OV],B00KDW2RSI


In [26]:
print(df.shape)
df['share_viewed'] = df['seconds_viewed'] / df['content_length_seconds_total']
df.head()

(6005798, 8)


,UserID,MovieID,content_type,content_length_seconds_total,pause_count,seconds_viewed,seconds_paused,offer_group_desc,share_viewed
0,A13GSUGTAOIT81,B00FZITHLK,Movie EST,14880,9,6949.248,2723.0,RENTAL,0.467019
1,A1965EQ7GVIP0Q,B01MS3SLVH,Movie EST,23760,6,6100.384,451.0,RENTAL,0.256750
2,A1XG9CQ2NVDJMP,B00ILFE8W2,Movie EST,25200,3,3925.284,12.0,RENTAL,0.155765
3,A23DRS92C2SITH,B00IK6JQSI,Movie EST,33120,1,7398.223,7.0,RENTAL,0.223376
4,A3HLHTE3XT5MDJ,B01MXTSU33,Movie EST,13920,11,9996.352,1114.0,RENTAL,0.718129


## Create ratings

In [27]:
df = df.dropna(axis = 0)

df['Rating'] = 0
df.loc[df['share_viewed'] <= 0.20, 'Rating'] = 1
df.loc[(df['share_viewed'] > 0.20) & (df['share_viewed'] <= 0.40), 'Rating'] = 2
df.loc[(df['share_viewed'] > 0.40) & (df['share_viewed'] <= 0.60), 'Rating'] = 3
df.loc[(df['share_viewed'] > 0.60) & (df['share_viewed'] <= 0.80), 'Rating'] = 4
df.loc[(df['share_viewed'] > 0.80), 'Rating'] = 5

In [28]:
df.head()

,UserID,MovieID,content_type,content_length_seconds_total,pause_count,seconds_viewed,seconds_paused,offer_group_desc,share_viewed,Rating
0,A13GSUGTAOIT81,B00FZITHLK,Movie EST,14880,9,6949.248,2723.0,RENTAL,0.467019,3
1,A1965EQ7GVIP0Q,B01MS3SLVH,Movie EST,23760,6,6100.384,451.0,RENTAL,0.256750,2
2,A1XG9CQ2NVDJMP,B00ILFE8W2,Movie EST,25200,3,3925.284,12.0,RENTAL,0.155765,1
3,A23DRS92C2SITH,B00IK6JQSI,Movie EST,33120,1,7398.223,7.0,RENTAL,0.223376,2
4,A3HLHTE3XT5MDJ,B01MXTSU33,Movie EST,13920,11,9996.352,1114.0,RENTAL,0.718129,4


In [29]:
df['Rating'].unique()

array([3, 2, 1, 4, 5], dtype=int64)

## Pivot the matrix, make subset 

In [30]:
ratings_df = df[['UserID','MovieID','Rating']].sample(frac = 0.005)

In [31]:
print(ratings_df.shape)
ratings_df.head()

(25244, 3)


,UserID,MovieID,Rating
5666100,A3QPUK1L9A7VX1,B01KUYZ6TU,4
5945632,A12D6OQRNM9VFQ,B00LUV8MT4,1
4220923,A3JZ1R7HW0LVUS,B06XRKG44W,2
5789771,A3OFKWTWFBI1H1,B06ZZ3PHW2,5
745230,AJSJQLN2BQ3FR,B017T27KT0,1


In [32]:
R_df = ratings_df.pivot(index = 'UserID', columns ='MovieID', values = 'Rating').fillna(0)
print(R_df.shape)
R_df.head()

(24730, 3701)


MovieID,B00EQXEKJ4,B00EQXEMZ6,B00EQXET7C,B00EQXF7P0,B00EQXFXZ4,B00EQY9R1Y,B00EQYTT9O,B00EQYTZZC,B00EQYU6B4,B00EQYU9B6,...,B078PK8KXT,B078PQ66SJ,B078RNXD4J,B078WF7WHZ,B078WFC9H9,B078WFJLY8,B078WFS1SX,B078WG5TT5,B078YTVBP4,B0797BPKL2
UserID,,,,,,,,,,,,,,,,,,,,,
A00741971H4WP7QFRBH5R,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A01208251IJET7J9A6T44,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A0131424ZO56580LL66D,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A01382682KM01R4KOU7A4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A0145872ZZSJCOXXPIE3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [33]:
#df.to_pickle(R_df)
R_df.shape

(24730, 3701)

In [34]:
R_df.head()

MovieID,B00EQXEKJ4,B00EQXEMZ6,B00EQXET7C,B00EQXF7P0,B00EQXFXZ4,B00EQY9R1Y,B00EQYTT9O,B00EQYTZZC,B00EQYU6B4,B00EQYU9B6,...,B078PK8KXT,B078PQ66SJ,B078RNXD4J,B078WF7WHZ,B078WFC9H9,B078WFJLY8,B078WFS1SX,B078WG5TT5,B078YTVBP4,B0797BPKL2
UserID,,,,,,,,,,,,,,,,,,,,,
A00741971H4WP7QFRBH5R,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A01208251IJET7J9A6T44,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A0131424ZO56580LL66D,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A01382682KM01R4KOU7A4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A0145872ZZSJCOXXPIE3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [35]:
t = R_df.reset_index()

In [36]:
t.head()

MovieID,UserID,B00EQXEKJ4,B00EQXEMZ6,B00EQXET7C,B00EQXF7P0,B00EQXFXZ4,B00EQY9R1Y,B00EQYTT9O,B00EQYTZZC,B00EQYU6B4,...,B078PK8KXT,B078PQ66SJ,B078RNXD4J,B078WF7WHZ,B078WFC9H9,B078WFJLY8,B078WFS1SX,B078WG5TT5,B078YTVBP4,B0797BPKL2
0,A00741971H4WP7QFRBH5R,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,A01208251IJET7J9A6T44,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,A0131424ZO56580LL66D,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,A01382682KM01R4KOU7A4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,A0145872ZZSJCOXXPIE3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Denorm the data, normalize by each users mean

In [37]:
R = R_df.as_matrix()
user_ratings_mean = np.mean(R, axis = 1)
R_demeaned = R - user_ratings_mean.reshape(-1, 1)

## SVD

In [38]:
from scipy.sparse.linalg import svds
U, sigma, Vt = svds(R_demeaned, k = 50)

In [39]:
sigma = np.diag(sigma)

In [40]:
all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) + user_ratings_mean.reshape(-1, 1)
preds_df = pd.DataFrame(all_user_predicted_ratings, columns = R_df.columns)

In [41]:
print(preds_df.shape)
preds_df.head()

(24730, 3701)


MovieID,B00EQXEKJ4,B00EQXEMZ6,B00EQXET7C,B00EQXF7P0,B00EQXFXZ4,B00EQY9R1Y,B00EQYTT9O,B00EQYTZZC,B00EQYU6B4,B00EQYU9B6,...,B078PK8KXT,B078PQ66SJ,B078RNXD4J,B078WF7WHZ,B078WFC9H9,B078WFJLY8,B078WFS1SX,B078WG5TT5,B078YTVBP4,B0797BPKL2
0,0.001370,0.001370,0.001370,0.001370,0.001370,0.001370,0.001370,0.001370,0.001370,0.001370,...,0.001370,0.001370,0.001370,0.001370,0.001370,0.001370,0.001370,0.001370,0.001371,0.001370
1,0.000008,0.000008,0.000011,0.000013,0.000010,0.000011,0.000008,0.000009,0.000010,0.000011,...,0.000003,0.000008,0.000008,0.000011,0.000012,0.000008,0.000007,0.000008,-0.000007,0.000008
2,0.000825,0.000825,0.000825,0.000825,0.000825,0.000825,0.000825,0.000825,0.000825,0.000825,...,0.000825,0.000825,0.000825,0.000825,0.000825,0.000825,0.000825,0.000825,0.000826,0.000825
3,0.001109,0.001109,0.001109,0.001108,0.001109,0.001109,0.001109,0.001109,0.001109,0.001109,...,0.001109,0.001109,0.001109,0.001109,0.001108,0.001109,0.001109,0.001109,0.001110,0.001109
4,0.001369,0.001369,0.001369,0.001369,0.001369,0.001369,0.001369,0.001369,0.001369,0.001369,...,0.001370,0.001369,0.001369,0.001369,0.001369,0.001369,0.001370,0.001369,0.001370,0.001369


In [42]:
new_pred = pd.concat([t['UserID'],preds_df],axis = 1)
new_pred = new_pred.set_index("UserID")
new_pred.head()

,B00EQXEKJ4,B00EQXEMZ6,B00EQXET7C,B00EQXF7P0,B00EQXFXZ4,B00EQY9R1Y,B00EQYTT9O,B00EQYTZZC,B00EQYU6B4,B00EQYU9B6,...,B078PK8KXT,B078PQ66SJ,B078RNXD4J,B078WF7WHZ,B078WFC9H9,B078WFJLY8,B078WFS1SX,B078WG5TT5,B078YTVBP4,B0797BPKL2
UserID,,,,,,,,,,,,,,,,,,,,,
A00741971H4WP7QFRBH5R,0.001370,0.001370,0.001370,0.001370,0.001370,0.001370,0.001370,0.001370,0.001370,0.001370,...,0.001370,0.001370,0.001370,0.001370,0.001370,0.001370,0.001370,0.001370,0.001371,0.001370
A01208251IJET7J9A6T44,0.000008,0.000008,0.000011,0.000013,0.000010,0.000011,0.000008,0.000009,0.000010,0.000011,...,0.000003,0.000008,0.000008,0.000011,0.000012,0.000008,0.000007,0.000008,-0.000007,0.000008
A0131424ZO56580LL66D,0.000825,0.000825,0.000825,0.000825,0.000825,0.000825,0.000825,0.000825,0.000825,0.000825,...,0.000825,0.000825,0.000825,0.000825,0.000825,0.000825,0.000825,0.000825,0.000826,0.000825
A01382682KM01R4KOU7A4,0.001109,0.001109,0.001109,0.001108,0.001109,0.001109,0.001109,0.001109,0.001109,0.001109,...,0.001109,0.001109,0.001109,0.001109,0.001108,0.001109,0.001109,0.001109,0.001110,0.001109
A0145872ZZSJCOXXPIE3,0.001369,0.001369,0.001369,0.001369,0.001369,0.001369,0.001369,0.001369,0.001369,0.001369,...,0.001370,0.001369,0.001369,0.001369,0.001369,0.001369,0.001370,0.001369,0.001370,0.001369


## Making movie recommendations

In [81]:
def recommend_movies(customerID, num_recommendations=3):

    sorted_user_predictions = new_pred.loc[customerID].sort_values(ascending=False)
    user_data = ratings_df[ratings_df['UserID'] == customerID]
    user_full = user_data.merge(movies_df,left_on="MovieID",
                            right_on="MovieID",how ="left").sort_values(['Rating'], ascending = False)
    
    movies_not_watched = movies_df[ ~movies_df['MovieID'].isin(user_full['MovieID'])]
    sorted_user_preds = pd.DataFrame(sorted_user_predictions.reset_index())
    sorted_user_preds.columns = ['MovieID','predictions']
    recommendation = pd.merge(left = movies_not_watched,
                              right = sorted_user_preds, left_on="MovieID",right_on="MovieID",how = 'left')
    recommendation = recommendation.sort_values(by = 'predictions', ascending = False)[0:num_recommendations]
    
    return user_full, recommendation

In [82]:
x,y = recommend_movies('A00741971H4WP7QFRBH5R',4)
y

,Title,MovieID,predictions
10417,Der Hobbit: Die Schlacht der fünf Heere [dt./OV],B00W4Z7EYO,0.001475
5479,Schöne Bescherung,B00HX1BO7W,0.001451
3278,Honig im Kopf,B0117VRVRU,0.001421
10384,Pitch Perfect 2 [dt./OV],B014PIFLXI,0.001417
